# Imports

In [1]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

## Functions

In [2]:
def get_car_locations(frame):

    # Find contours in the binary image
    contours, _ = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(frame, contours, -1, (0, 255, 0), 3)
    
    car_locations = []

    # Find the car location
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 500 and area < 1500:
            
            # Find the bounding rectangle of the car
            x, y, w, h = cv2.boundingRect(contour)
            car_locations.append((x, y, w, h))

    return car_locations

In [3]:
def get_road_boundaries(road, frame):
    
    # Find contours in the binary image
    contours, _ = cv2.findContours(road.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    road_locations = []
    
    # Find the roads sections
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 1000:
            
            cv2.drawContours(frame, contour, -1, (0, 0, 0), 2)

In [4]:
def get_road_and_background_real_time(video_path, output_width, output_height, kernel):

    cap = cv2.VideoCapture(video_path)
    
    # Reset video capture to process frames
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    # Define output video settings
    output_fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Initialize variables
    i = 0
    road = None
    all_frames = None
    frams_to_process = 2600
    frams_for_background = frams_to_process // 2

    # Calculate the road and background
    while cap.isOpened():
        ret, curr_frame = cap.read()

        if not ret:
            break
        
        # Convert curr_frame to grayscale
        curr_frame_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY).astype('float64')
        
        # Resize curr_frame to the desired output size
        curr_frame_gray = cv2.resize(curr_frame_gray, (output_width, output_height))
        
        # Build the background of the video
        if i == 0:
            all_frames = np.zeros((frams_for_background, *curr_frame.shape), dtype=curr_frame.dtype)
            
        if i < frams_for_background:
            all_frames[i] = curr_frame
            i += 1
            continue
        
        elif i == frams_for_background:
            background = np.uint8(np.round(np.median(all_frames, axis=0)))
            background = cv2.resize(background, (output_width, output_height))
            background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY).astype('float64')
            cv2.imshow('Background', background.astype('uint8'))
            cv2.waitKey(0)
            i += 1
            continue
        
        # Build the road
        if i == frams_for_background + 1:
            diff_frame = get_binary_frame(background, curr_frame_gray, kernel)
            road = diff_frame
            i += 1
            continue
            
        if i < frams_to_process:
            diff_frame = get_binary_frame(background, curr_frame_gray, kernel)
            road += diff_frame
            road = np.clip(road, 0, 255)
            road = cv2.normalize(road, None, 0, 255, cv2.NORM_MINMAX)
            road = np.round(road)
            i += 1

        elif i == frams_to_process:
            road = cv2.resize(road, (output_width, output_height))
            i += 1
            
        else:
            cap.release()
            cv2.destroyAllWindows()
            return road, background
        
        # Display the road in real-time 
        cv2.imshow('Road', road)
        cv2.waitKey(1)

In [5]:
def get_binary_frame(frame1, frame2, kernel):
    
    # Calculate the difference between the frames
    diff_frame = cv2.absdiff(frame1, frame2)
    
    # Create threshold
    th = 50
    diff_frame[diff_frame < th] = 0
    diff_frame[diff_frame >= th] = 255
    
    # Using morphological operations
    diff_frame = cv2.morphologyEx(diff_frame, cv2.MORPH_DILATE, kernel, iterations=3)
    diff_frame = cv2.morphologyEx(diff_frame, cv2.MORPH_OPEN, kernel, iterations=6)
    
    # Display diff_frame
    cv2.imshow('diff_frame', diff_frame)
    cv2.waitKey(1)
    
    return diff_frame

## Program

In [6]:
# Import the live traffic video
liveTrafficVideo = 'video4.mp4'
cap = cv2.VideoCapture(liveTrafficVideo)

# Reset video capture to process frames
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Define output video settings
output_width = 800  
output_height = 400  
output_fps = 30

# Create output video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_path = 'output_video.mp4'
out = cv2.VideoWriter(output_path, fourcc, output_fps, (output_width, output_height))

# Define kernel
kernel = np.ones((3, 3), np.uint8)

# Preprocessing the road and the background
road, background = get_road_and_background_real_time(liveTrafficVideo, output_width, output_height, kernel)

# Initialize variables
count = 0
center_pt_prev = []
tracking_objects = {}
tracking_id = 0

while cap.isOpened():
    ret, curr_frame = cap.read()
    
    if not ret:
        break
        
    count += 1
    
    # Convert curr_frame to grayscale
    curr_frame_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY).astype('float64')

    # Resize frames to the desired output size
    curr_frame = cv2.resize(curr_frame, (output_width, output_height))
    curr_frame_gray = cv2.resize(curr_frame_gray, (output_width, output_height))

    # Calculate the difference between the frames
    diff_frame = get_binary_frame(background, curr_frame_gray, kernel)
    
    # Define ROI
    height, width = output_height, output_width
    cars_roi = diff_frame[output_height // 2 :, :]
    road_roi = road[output_height // 2 :, :]

    # Calculate the padding that is needed
    pad_height = diff_frame.shape[0] - cars_roi.shape[0]
    pad_width = diff_frame.shape[1] - cars_roi.shape[1]

    # Pad the ROI with zeros to match the shape of diff_frame
    padded_cars_roi = np.pad(cars_roi, ((pad_height, 0), (0, pad_width)), mode='constant').astype('uint8')
    padded_road_roi = np.pad(road_roi, ((pad_height, 0), (0, pad_width)), mode='constant').astype('uint8')
    
    # Detect cars in ROI
    cars = get_car_locations(padded_cars_roi)
    
    # Detect road in ROI
    #get_road_boundaries(padded_road_roi, curr_frame)
    
    center_pt_curr = []

    # Find the center point in each car in the frame
    for (x, y, w, h) in cars:
        cx = int((x + x + w) / 2)
        cy = int((y + y + h) / 2)
        center_pt_curr.append((cx,cy))
        
        # The car is not on the road
        if road[cy, cx] == 0:
            cv2.rectangle(curr_frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            # Implement a function that sends an email to office@yedidim-il.org or SMS to 1230 with the curr_frame
            
        # The car on the road    
        else:
            cv2.rectangle(curr_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
         
    if count <= 2:    
        for pt in center_pt_curr:
            for pt2 in center_pt_prev:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                if distance < 20:
                    tracking_objects[tracking_id] = pt
                    tracking_id += 1
    else:
        tracking_objects_copy = tracking_objects.copy()
        center_pt_curr_copy = center_pt_curr.copy()
        for object_id, pt2 in tracking_objects_copy.items():
            object_exists = False
            for pt in center_pt_curr_copy:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])
                
                # Update IDs position
                if distance < 20:
                    tracking_objects[object_id] = pt
                    object_exists = True
                    if pt in center_pt_curr:
                        center_pt_curr.remove(pt)
                    continue
                    
            # Remove IDs lost
            if not object_exists:
                tracking_objects.pop(object_id)
                
        # Add new IDs found
        for pt in center_pt_curr:
            tracking_objects[tracking_id] = pt
            tracking_id += 1

    for object_id, pt in tracking_objects.items():
        cv2.circle(curr_frame, pt, 2, (255,255,255), -1)
        text = f"ID:{str(object_id)}"
        cv2.putText(curr_frame, text, (pt[0], pt[1] - 7), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,255,255), 1)
    
    # Display the frames
    cv2.imshow('Cars ROI', cars_roi)
    cv2.imshow('Diff_frames', diff_frame)
    cv2.imshow('Current frame', curr_frame.astype('uint8'))
    
    # Update the previous point
    center_pt_prev = center_pt_curr.copy()
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video
cap.release()
out.release()
cv2.destroyAllWindows()